1. Load the Dataset

In [2]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.1 MB ? eta -:--:--
   - -------------------------------------- 1.0/25.1 MB 2.6 MB/s eta 0:00:10
   -- ------------------------------------- 1.6/25.1 MB 2.9 MB/s eta 0:00:09
   --- ------------------------------------ 2.1/25.1 MB 2.7 MB/s eta 0:00:09
   --- ------------------------------------ 2.4/25.1 MB 2.6 MB/s eta 0:00:09
   ---- ----------------------------------- 2.9/25.1 MB 2.4 MB/s eta 0:00:10
   ----- ---------------------------------- 3.1/25.1 MB 2.3 MB/s eta 0:00:10
   ----- ---------------------------------- 3.7/25.1 MB 2.2 MB/s eta 0:00:10
   ------ --------------------------------- 3.9/25.1 MB 2.2 MB/s eta 0:00:10
   ------- -------------------------------- 4.5/25.1 MB 2.2 MB/s eta 0:00:10
   ------- -------------------------------- 4.7/25.1 MB 2.2 MB/s eta 0:00:10
   -------- 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Split the dataset into training and validation subsets
train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\datasets--SKNahin--bengali-transliteration-data. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adminis

2. Data Preprocessing

In [5]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.1 MB 1.0 MB/s eta 0:00:10
   --- ------------------------------------ 0.8/10.1 MB 1.0 MB/s eta 0:00:09
   ---- ----------------------------------- 1.0/10.1 MB 1.0 MB/s eta 0:00:09
   ---- ----------------------------------- 1.0/10.1 MB 1.0 MB/s eta 0:00:09
   ----- ---------------------------------- 1.3/10.1 MB 1.0 MB/s eta 0:00:09
   ------ --------------------------------- 1.6/10.1 MB 1.0 MB/s eta 0:00:09
   ------- -------------------------------- 1.8/10.1 MB 1.0 MB/s eta 0:00:08
   -------- ------------------------------- 2.1/10.1 MB 1.1 MB/s eta 0:00:08
   -------- ------------------------------- 2.1/10.1 MB 1.1 MB/s eta 0:00:08
   --------- ------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import AutoTokenizer

# Choose a tokenizer that matches the model we will use
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ROMANCE")


# Tokenize the datasets
def tokenize_function(examples):
    inputs = examples['rm']
    targets = examples['bn']
    model_inputs = tokenizer(
        inputs, 
        max_length=128, 
        truncation=True, 
        padding="max_length"  # Ensure padding to the max length
    )
    labels = tokenizer(
        targets, 
        max_length=128, 
        truncation=True, 
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

3. Select a Model

In [ ]:
from transformers import MBartForConditionalGeneration

# Load the pre-trained mBART model
model = MBartForConditionalGeneration.from_pretrained("google/byt5-small")

4. Train the Model

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()